In [1]:
import pandas as pd

In [2]:
combined = pd.read_csv('data/Combined_News_DJIA.csv')
news = pd.read_csv('data/RedditNews.csv')
market = pd.read_csv('data/upload_DJIA_table.csv')

In [3]:
print(combined.shape)
combined.head(1)

(1989, 27)


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


In [4]:
# Remove 'b'-prefixes
# Applymap -> Like .apply() but Applied to Entire DF
combined = combined.applymap(lambda cell: cell.strip() if type(cell)==str else cell)
combined = combined.applymap(lambda cell: cell.lstrip('b"') if type(cell)==str else cell)
combined.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...,'BREAKING: Musharraf to be impeached.','Russia Today: Columns of troops roll into Sou...,'Russian tanks are moving towards the capital ...,"Afghan children raped with 'impunity,' U.N. of...",'150 Russian tanks have entered South Ossetia ...,"Breaking: Georgia invades South Ossetia, Russi...",The 'enemy combatent' trials are nothing but a...,...,'Georgia Invades South Ossetia - if Russia get...,'Al-Qaeda Faces Islamist Backlash',"'Condoleezza Rice: ""The US would not act to pr...",'This is a busy day: The European Union has a...,"Georgia will withdraw 1,000 soldiers from Iraq...",'Why the Pentagon Thinks Attacking Iran is a B...,'Caucasus in crisis: Georgia invades South Oss...,'Indian shoe manufactory - And again in a ser...,'Visitors Suffering from Mental Illnesses Bann...,"No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,'Why wont America and Nato help us? If they wo...,'Bush puts foot down on Georgian conflict',Jewish Georgian minister: Thanks to Israeli tr...,'Georgian army flees in disarray as Russians a...,"Olympic opening ceremony fireworks 'faked'""",'What were the Mossad with fraudulent New Zeal...,'Russia angered by Israeli military sale to Ge...,'An American citizen living in S.Ossetia blame...,...,'Israel and the US behind the Georgian aggress...,"'""Do not believe TV, neither Russian nor Georg...",'Riots are still going on in Montreal (Canada)...,'China to overtake US as largest manufacturer','War in South Ossetia [PICS]','Israeli Physicians Group Condemns State Torture',' Russia has just beaten the United States ove...,'Perhaps *the* question about the Georgia - Ru...,'Russia is so much better at war',So this is what it's come to: trading sex for ...


In [5]:
# Sample Corpus from Combined DF
# -- Includes Only Top1 Headlines
corpus = combined[['Date', 'Label', 'Top1']]
print(corpus.shape)
corpus.head(3)

(1989, 3)


,Date,Label,Top1
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...
1,2008-08-11,1,'Why wont America and Nato help us? If they wo...
2,2008-08-12,0,'Remember that adorable 9-year-old who sang at...


In [13]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

## Try on Built-In Dataset

In [14]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [15]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
rfc = RandomForestClassifier()

In [16]:
# Define the Pipeline

pipe = Pipeline([
    # Vectorizer
    ('vect', vect),
    # Classifier
    ('clf', rfc)
])


In [22]:
# Potential Parameters for Our Pipeline Components
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500, 1000),
    'clf__max_depth': (15, 20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=6, verbose=1)
# grid_search.fit(X, y)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.6s
[Parallel(n_jobs=6)]: Done  80 out of  80 | elapsed:   12.0s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=6,
             param_grid={'clf__max_depth': (15, 20),
                         'vect__max_df': (0.75, 1.0),
                         'vect__max_features': (500, 1000),
                         'vect__min_df': (0.02, 0.05)},
             verbose=1)

In [35]:
# grid_search.best_params_, grid_search.best_score_
# ({'clf__max_depth': 20,
#   'vect__max_df': 1.0,
#   'vect__max_features': 1000,
#   'vect__min_df': 0.02},
#  0.9089827281381749)

## Try with Headline Data

In [27]:
print(corpus.shape)
corpus.head(3)

(1989, 3)


,Date,Label,Top1
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...
1,2008-08-11,1,'Why wont America and Nato help us? If they wo...
2,2008-08-12,0,'Remember that adorable 9-year-old who sang at...


In [28]:
# Same Pipeline Components
# Text to Vector and Classifier
vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
rfc = RandomForestClassifier()

In [29]:
# Define the Pipeline

pipe = Pipeline([
    # Vectorizer
    ('vect', vect),
    # Classifier
    ('clf', rfc)
])

In [32]:
# Potential Parameters for Our Pipeline Components
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500, 1000),
    'clf__max_depth': (15, 20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=6, verbose=1)
# grid_search.fit(X, y)
grid_search.fit(corpus['Top1'], corpus['Label'])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  80 out of  80 | elapsed:    3.4s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=6,
             param_grid={'clf__max_depth': (15, 20),
                         'vect__max_df': (0.75, 1.0),
                         'vect__max_features': (500, 1000),
                         'vect__min_df': (0.02, 0.05)},
             verbose=1)

In [34]:
# grid_search.best_params_, grid_search.best_score_
# ({'clf__max_depth': 15,
#   'vect__max_df': 1.0,
#   'vect__max_features': 500,
#   'vect__min_df': 0.05},
#  0.5369618875232586)

### Try with spaCy Vectorizor


In [45]:
import spacy
import numpy as np
nlp = spacy.load('en_core_web_lg')
vectors = [np.array(nlp(word).vector) for word in corpus['Top1']]

In [55]:
#vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
def vectorize(text):
    return nlp(text).vector

In [61]:
help(nlp.Defaults)

Help on class EnglishDefaults in module spacy.lang.en:

class EnglishDefaults(spacy.language.BaseDefaults)
 |  Method resolution order:
 |      EnglishDefaults
 |      spacy.language.BaseDefaults
 |      builtins.object
 |  
 |  Class methods defined here:
 |  
 |  is_base_form(univ_pos, morphology=None) from builtins.type
 |      Check whether we're dealing with an uninflected paradigm, so we can
 |      avoid lemmatization entirely.
 |      
 |      univ_pos (unicode / int): The token's universal part-of-speech tag.
 |      morphology (dict): The token's morphological features following the
 |          Universal Dependencies scheme.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  lex_attr_getters = {1: <function is_alpha>, 2: <function is_ascii>, 3:...
 |  
 |  morph_rules = {'IN': {'Albeit': {'POS': 'SCONJ'}, 'Although': {'POS': ...
 |  
 |  paired_orth_variants = [{'tags': ['``', "''"], 'variants':

In [57]:
# We Just Need a Classifier as the text has already been classified

pipe = Pipeline([
    # Vectorizer
    ('vect', vectorize),
    # Classifier
    ('clf', rfc)
])

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' '<function vectorize at 0x7fd514ed1430>' (type <class 'function'>) doesn't

In [50]:
# Potential Parameters for Our Pipeline Components
parameters = {
    'clf__max_depth': (15, 20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=6, verbose=1)
# grid_search.fit(X, y)
grid_search.fit(corpus['Vectors'], corpus['Label'])

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    0.2s finished


ValueError: setting an array element with a sequence.